In [1]:
import pandas as pd
import os
from datetime import datetime
import utils.data_utils as data_utils
from collections import defaultdict

LONG_TERM_ACTIONS = ['walking', 'eating', 'smoking', 'discussion']
ACTIONS = data_utils.define_actions('all')
ACTIONS.sort(key=len)

In [4]:
def csv_iter():
    for f in os.listdir('./checkpoint/test'):
        if not f.startswith('short_term'):
            continue
        try:
            datetime.strptime(f[-23:-4], "%d-%m-%Y-%H:%M:%S")
            if f.split('.')[-1] == 'csv':
                day = int(f[-23:-21])
                if day > 20:
                    yield f
        except:
            pass

def extend_df(df, long_term=True):
    if long_term:
        actions = LONG_TERM_ACTIONS
        time_lens = [560, 1000]
    else:
        actions = ACTIONS
        time_lens = [80, 160, 320, 400]
    columns_mapping = defaultdict(list)
    for time_len in time_lens:
        for action_name in actions:
            columns_mapping['3d' + str(time_len)].append(action_name + '3d' + str(time_len))
            columns_mapping['usingfulltestset_3d' + str(time_len)].append('usingfulltestset_' + action_name + '3d' + str(time_len))

    for key in columns_mapping:
        df[key] = df[columns_mapping[key]].mean(axis=1)
    
    return list(columns_mapping.keys())

def get_summary_df(merged):
    cols = list(set([i[:-5] for i in merged.columns]))
    new_cols = []
    for col in cols:
        col_name = [c for c in merged.columns if c.startswith(col)]
        new_cols.append(col+'_mean')
        new_cols.append(col+'_std')
        merged[col+'_mean'] = merged[col_name].mean(axis=1)
        merged[col+'_std'] = merged[col_name].std(axis=1)
    return merged[new_cols]

In [17]:
df_lst = []
mini_df_lst = []
for i, f in enumerate(csv_iter()):
    print(i, f)
    df = pd.read_csv(os.path.join('checkpoint/test', "./", f))
    df = df.sort_values(by='v_3d')
    df_lst.append(df[extend_df(df, long_term=False)].add_suffix('_run' + str(i)))
    mini_df_lst.append(df[extend_df(df, long_term=False)].head(1).add_suffix('_run' + str(i)).reset_index(drop=True))

merged = pd.concat(mini_df_lst, axis=1)

0 short_termmain_3d_3D_in10_out10_dct_n_30_28-06-2020-17:11:11.csv
1 short_termmain_3d_3D_in10_out10_dct_n_30_26-06-2020-17:51:18.csv
2 short_termmain_3d_3D_in10_out10_dct_n_30_27-06-2020-05:36:39.csv
3 short_termmain_3d_3D_in10_out10_dct_n_30_27-06-2020-17:23:34.csv
4 short_termmain_3d_3D_in10_out10_dct_n_30_28-06-2020-05:20:17.csv


In [18]:
tmp = get_summary_df(merged)
tmp[[i for i in tmp.columns if not i.startswith('usingfulltestset')]]

,3d320_mean,3d320_std,3d400_mean,3d400_std,3d80_mean,3d80_std,3d160_mean,3d160_std
0,50.417804,0.342099,60.950869,0.46302,11.416297,0.168276,24.375663,0.286886


In [8]:
mini_df_lst[0]

,3d80_run0,usingfulltestset_3d80_run0,3d160_run0,usingfulltestset_3d160_run0,3d320_run0,usingfulltestset_3d320_run0,3d400_run0,usingfulltestset_3d400_run0
0,11.456757,10.540577,24.508205,23.004704,50.59056,48.326132,61.277202,59.85636


In [9]:
mini_df_lst[1]

,3d80_run1,usingfulltestset_3d80_run1,3d160_run1,usingfulltestset_3d160_run1,3d320_run1,usingfulltestset_3d320_run1,3d400_run1,usingfulltestset_3d400_run1
0,11.567771,10.609106,24.594277,23.089859,50.895493,48.384029,61.551733,59.84307


In [10]:
mini_df_lst[2]

,3d80_run2,usingfulltestset_3d80_run2,3d160_run2,usingfulltestset_3d160_run2,3d320_run2,usingfulltestset_3d320_run2,3d400_run2,usingfulltestset_3d400_run2
0,11.130844,10.489414,23.87558,22.955754,50.001742,48.325465,60.443906,59.766395


In [11]:
mini_df_lst[3]

,3d80_run3,usingfulltestset_3d80_run3,3d160_run3,usingfulltestset_3d160_run3,3d320_run3,usingfulltestset_3d320_run3,3d400_run3,usingfulltestset_3d400_run3
0,11.425437,10.514747,24.486741,22.860766,50.231618,47.883244,60.591237,59.326504


In [12]:
mini_df_lst[4]

,3d80_run4,usingfulltestset_3d80_run4,3d160_run4,usingfulltestset_3d160_run4,3d320_run4,usingfulltestset_3d320_run4,3d400_run4,usingfulltestset_3d400_run4
0,11.500675,10.676572,24.413511,23.155687,50.369607,48.408978,60.890268,59.925512
